# Packages

In [ ]:
import os
import sys
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
sys.path.insert(0, '..')
import numpy as np
from PIL import Image
import os

# Load images in Cifar-100 & Eval500 

In [3]:
device = "cuda"

eval500_folder = "/home/adl/Desktop/zenn/SPML/SPML_HW1/cifar-100_eval/"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])
tensor_images = []
file_names = []
for filename in os.listdir(eval500_folder):
    if filename.endswith('.png'):  
        image_path = os.path.join(eval500_folder, filename)
        image = Image.open(image_path).convert('RGB')  
        tensor_image = transform(image).to(device)  
        tensor_images.append(tensor_image)  
        file_names+=[filename]
        
cifar100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
cifar100loader = DataLoader(cifar100, batch_size=64, shuffle=False)


Files already downloaded and verified


# Compare images in Cifar-100 & Eval500 and safe the labels 

In [ ]:
original_images = []
original_labels = []
for data in cifar100loader:
              images, labels = data
              images, labels = images.to(device), labels.to(device)  # Move data to the same device as the model)
              images.requires_grad = True   
              original_images += images
              original_labels += labels
              
eval_labels = []

for i in range(len(tensor_images)):
    for j in range(0,len(original_images)):
        if torch.equal(tensor_images[i],original_images[j]):
            eval_labels += [original_labels[j].unsqueeze(0)]

## Export the labels

In [ ]:
eval_labels_save = []
for i in range(len(eval_labels)):
    eval_labels_save.append(eval_labels[i].cpu().numpy())
np.save('labels.npy', eval_labels_save)